In [87]:
# %matplotlib widget
import json
from matplotlib import pyplot as plt
import numpy as np
from itertools import product

In [88]:
host_l = [ "niagara", "nile", "kolyma", "euphrates", ]
site_l = [ "local", "nfs" ]
bs_l = [ "4k", "16k", "64k", "256k", "1m", ]
jobs_l = [ 1, 2, 4, 8, 16, ]
fsize_l = [ "1G", "5G"]

In [89]:
def create_plot(host, site, bs, jobs, fsize, figsize=(10, 5)):
    modes = [ "rand_write", "rand_read", "seq_write", "seq_read", "mixed_rw" ]
    plot_data = {}
    host = [host] if isinstance(host, list) is False else host
    site = [site] if isinstance(site, list) is False else site
    bs = [bs] if isinstance(bs, list) is False else bs
    jobs = [jobs] if isinstance(jobs, list) is False else jobs
    file_size = [fsize] if isinstance(fsize, list) is False else fsize
    for h, s, b, j, f in product(host, site, bs, jobs, file_size):
        key = f"{h}_{s}_bs{b}_jobs{j}_size{f}"
        plot_data[key] = {}
        for m in modes:
            with open(f"results/{h}_{s}_bs{b}_jobs{j}_size{f}_{m}.txt") as fd:
                # print(fd)
                data = json.load(fd)
                if m.endswith("write"):
                    plot_data[key][m] = float(data["jobs"][0]["write"]["bw"])/1024
                elif m.endswith("read"):
                    plot_data[key][m] = float(data["jobs"][0]["read"]["bw"])/1024
                else:
                    plot_data[key]["mixed_read"] = float(data["jobs"][0]["read"]["bw"])/1024
                    plot_data[key]["mixed_write"] = float(data["jobs"][0]["write"]["bw"])/1024

    # print(plot_data)

    fig, ax = plt.subplots(ncols=len(list(plot_data.keys())), layout="constrained", figsize=figsize, sharey=True, squeeze=False)
    ax = ax.flatten()
    for k, (pname, pdata) in enumerate(plot_data.items()):
        width = 0.25
        x = np.arange(3)*0.6
        seqrand = ["Sequential", "Random", "Mixed"]

        name = {
            "Read": ["seq_read", "rand_read", "mixed_read"],
            "Write": ["seq_write", "rand_write", "mixed_write"],
        }

        for i, (key, value) in enumerate(name.items()):
            rects = ax[k].bar(x + i*width, [pdata[v] for v in value], width=width, label=key)
            ax[k].bar_label(rects, padding=3)

        ax[k].set_title(pname)
        ax[k].set_xticks(x + width/2, seqrand)
        ax[k].legend(loc='best', ncols=3)
        # ax[k].set_ylim(0)
        # set horizontal grid
        ax[k].yaxis.grid(True)
    ax[0].set_ylabel('Bandwidth (MB/s)')

    fig.show()

# create_graph("niagara", site="local", bs="4k", jobs=1, fsize="1G")
# create_graph("niagara", site="local", bs=bs_l, jobs=1, fsize="1G", figsize=(20, 5))
# create_graph("niagara", site="local", bs=bs_l, jobs=1, fsize="5G", figsize=(20, 5))
# create_plot("niagara", site="local", bs="4k", jobs=jobs_l, fsize="5G", figsize=(20, 5))
# create_plot("niagara", site="nfs", bs="4k", jobs=jobs_l, fsize="5G", figsize=(20, 5))
# create_plot("nile", site="nfs", bs="4k", jobs=jobs_l, fsize="5G", figsize=(20, 5))

# create_graph("niagara", "nfs", bs, jobs, fsize_l)
# create_graph("niagara", "nfs", bs_l, jobs, fsize, figsize=(20, 5))
# create_graph("niagara", "nfs", bs_l, jobs, "5G", figsize=(20, 5))
# create_graph("kolyma", site, bs, jobs, fsize)
# create_graph("nile", site, bs, jobs, fsize)
# create_graph("euphrates", site, bs, jobs, fsize)

Um desktop típico do Centro Pi: niagara

Usando o SSD (local), variando o tamanho dos blocos usados {4k, 16k, 256k, 1m}, com 1 processo (jobs), usando um de 5G dados.

In [ ]:
create_plot("niagara", site="local", bs=bs_l, jobs=1, fsize="5G", figsize=(20, 5))

* Leitura sequêncial melhora com o aumento do tamanho do bloco. (Qual é o limite?)
* Escrita sequêncial varia muito pouco, porém tem um "sweet spot" com blocks de 16k. Talvez isso seja alguma característica do device, como o tamanho do block em disco.
* Leitura e escrita aleatória diminuem muito a performance do SSD. Porém a escrita é mais rápida, talvez pelo buffer do SSD.
* Leitura/Escrita misturadas tem o pior desempenho, mas representam melhor o caso de uso médio para um desktop.

O mesmo teste usando arquivos de 1GB não mostra variação relevante.

In [ ]:
create_plot("niagara", site="local", bs=bs_l, jobs=1, fsize="1G", figsize=(20, 5))

Fixando o tamanho do block em 4k e variando o número de jobs para simular "usuários" concorrentes.

Os valores abaixo já estão agrupados, ié, os valores são a soma dos jobs individuais.

In [ ]:
create_plot("niagara", site="local", bs="4k", jobs=jobs_l, fsize="5G", figsize=(20, 5))

* A leitura sequencial cai muito. Talvez por deixar de ser sequencial, o que leva a operação para perto da leitura aleatória.
* A escrita até 4 jobs melhora.
* A leitura e escrita mista se mantem nos mesmos patamares.

Para comparação: o mesmo teste acima com outros tamanhos de bloco. (Alguns testes estão com problema!)

In [ ]:
create_plot("niagara", site="local", bs="64k", jobs=jobs_l, fsize="5G", figsize=(20, 5))
create_plot("niagara", site="local", bs="256k", jobs=jobs_l, fsize="5G", figsize=(20, 5))
create_plot("niagara", site="local", bs="1m", jobs=jobs_l, fsize="5G", figsize=(20, 5))

* Aumentar o block tem efeito positivo na leitura sequencial
* Também melhorar a escrita/leitura mista.
* Parece piorar para o caso aleatório.

Repetindo os teste para o niagara, porém usando o volume em NFS.

Neste caso a rede foi deixada totalmente ociosa e nenhum outro processo acessava a euphrates.

In [ ]:
create_plot("niagara", site="nfs", bs=bs_l, jobs=1, fsize="5G", figsize=(20, 5))

Variando o número de acesso concorrentes:

In [ ]:
create_plot("niagara", site="nfs", bs="4k", jobs=jobs_l, fsize="5G", figsize=(20, 5))

* Leitura sequencia e aleatória são limitadas a 33MB/s para um job e sobem até 108MB/s, que é o limite da rede para a niagara, com mais jobs. Isto indica uma limitação do processo de NFS que tem throughput máxido de 33MB/s, aparentemente.
* Leitura/Escrita misturada são muito baixas para blocos de 4kB, considerando que temos apenas 1 job. Melhor com mais jobs porém é possivel ver que o máximo de desempenho se antes (jobs < 16)

Outros tamanhos de bloco:

In [ ]:
create_plot("niagara", site="nfs", bs="64k", jobs=jobs_l, fsize="5G", figsize=(20, 5))
create_plot("niagara", site="nfs", bs="256k", jobs=jobs_l, fsize="5G", figsize=(20, 5))
create_plot("niagara", site="nfs", bs="1m", jobs=jobs_l, fsize="5G", figsize=(20, 5))

* Aumentar o tamanho do bloco melhora o desempenho de leitura
* Leitura aleatória ainda fica em ~33MB/s. Limitação de CPU para o NFS?
* Estes gráfico sugerem que usar blocos de 64kB otimizaria o servidor.


Vamos repetir a análise para a nile. Nile tem interface de 10Gb/s (1125MB/s), portanto os testes devem passar de 112MB/s vistos acima.

In [ ]:
create_plot("nile", site="nfs", bs=bs_l, jobs=1, fsize="5G", figsize=(20, 5))

In [ ]:
create_plot("nile", site="nfs", bs="64k", jobs=jobs_l, fsize="5G", figsize=(20, 5))
create_plot("nile", site="nfs", bs="256k", jobs=jobs_l, fsize="5G", figsize=(20, 5))
create_plot("nile", site="nfs", bs="1m", jobs=jobs_l, fsize="5G", figsize=(20, 5))

* Neste caso dá para ver como a rede interfere nos números de leitura, chegando a > 1GB/s para bs de 1m e 4 jobs.
* Uma dúvida é porque os casos de leitura/escrita misturadas são maiores que leitura e escrita aleatórias.

Para comparação, estes são os dados da euphrates rodando os mesmos testes localmente (sem NFS)

In [ ]:
create_plot("euphrates", site="local", bs=bs_l, jobs=1, fsize="5G", figsize=(20, 5))

In [ ]:
create_plot("euphrates", site="local", bs="64k", jobs=jobs_l, fsize="5G", figsize=(20, 5))
create_plot("euphrates", site="local", bs="256k", jobs=jobs_l, fsize="5G", figsize=(20, 5))
create_plot("euphrates", site="local", bs="1m", jobs=jobs_l, fsize="5G", figsize=(20, 5))

## Euphrates
* Duas controladoras Apolo4510
    *  /dev/sda é um RAID5 (?) de 260TB 
    * /dev/sdc é um RAID5 (?) de 180TB
* /dev/md0 é um RAID 0 em software onde está /impa/home ...